In [18]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow

In [19]:
!pip show azure-cognitiveservices-vision-computervision

Name: azure-cognitiveservices-vision-computervision
Version: 0.9.0
Summary: Microsoft Azure Cognitive Services Computer Vision Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azpysdkhelp@microsoft.com
License: MIT License
Location: /Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages
Requires: azure-common, msrest
Required-by: 


In [20]:
import sys
sys.path

['/Users/gonakazawa/Documents/go/dev_environments/02_物体検出アプリ',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages',
 '/opt/homebrew/opt/six/lib/python3.10/site-packages',
 '/opt/homebrew/lib/python3.10/site-packages']

In [21]:
sys.path.append("/Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages")

In [62]:
import sys
sys.path

['/Users/gonakazawa/Documents/go/dev_environments/02_物体検出アプリ',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages',
 '/opt/homebrew/opt/six/lib/python3.10/site-packages',
 '/opt/homebrew/lib/python3.10/site-packages',
 '/Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages']

In [63]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

import json

In [64]:
with open('secret.json') as f:
    secret = json.load(f)

In [65]:
KEY = secret['KEY']
ENDPOINT = secret['ENDPOINT']

In [66]:
computervision_client = ComputerVisionClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [67]:
remote_image_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/landmark.jpg"

In [68]:
print("===== Describe an image - remote =====")
description_results = computervision_client.describe_image(remote_image_url)

print("Description of remote image: ")
if (len(description_results.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

===== Describe an image - remote =====
Description of remote image: 
'an ancient city with many ruins with Colosseum in the background' with confidence 33.80%


## GET IMAGE CATEGOLIES

In [31]:
print("===== Categorize an image - remote =====")
remote_image_features = ["categories"]
categorize_results_remote = computervision_client.analyze_image(remote_image_url , remote_image_features)

print("Categories from remote image: ")
if (len(categorize_results_remote.categories) == 0):
    print("No categories detected.")
else:
    for category in categorize_results_remote.categories:
        print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))

===== Categorize an image - remote =====
Categories from remote image: 
'building_' with confidence 31.64%
'others_' with confidence 0.39%
'outdoor_' with confidence 3.91%


## GET IMAGE TAGS

In [69]:
print("===== Tag an image - remote =====")
tags_result_remote = computervision_client.tag_image(remote_image_url )

print("Tags in the remote image: ")
if (len(tags_result_remote.tags) == 0):
    print("No tags detected.")
else:
    for tag in tags_result_remote.tags:
        print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))

===== Tag an image - remote =====
Tags in the remote image: 
'outdoor' with confidence 99.00%
'building' with confidence 98.81%
'sky' with confidence 98.21%
'stadium' with confidence 98.17%
'ancient rome' with confidence 96.16%
'ruins' with confidence 95.04%
'amphitheatre' with confidence 93.99%
'ancient roman architecture' with confidence 92.65%
'historic site' with confidence 89.55%
'ancient history' with confidence 89.54%
'history' with confidence 86.72%
'archaeological site' with confidence 84.41%
'travel' with confidence 65.85%
'large' with confidence 61.02%
'city' with confidence 56.57%


## DETECT OBJECTS

In [70]:
print("===== Detect Objects - remote =====")
remote_image_url_objects = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/objects.jpg"
detect_objects_results_remote = computervision_client.detect_objects(remote_image_url_objects)

print("Detecting objects in remote image:")
if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results_remote.objects:
        print("object at location {}, {}, {}, {}".format( \
        object.rectangle.x, object.rectangle.x + object.rectangle.w, \
        object.rectangle.y, object.rectangle.y + object.rectangle.h))

===== Detect Objects - remote =====
Detecting objects in remote image:
object at location 213, 365, 85, 208
object at location 218, 402, 179, 384
object at location 238, 417, 298, 416
object at location 116, 419, 60, 386


## READ LOCAL FILES

In [71]:
local_image_path = 'sample05.jpg'

local_image = open(local_image_path, "rb")

print("===== Detect Objects - local =====")
detect_objects_results = computervision_client.detect_objects_in_stream(local_image)

print("Detecting objects in local image:")
if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results.objects:
        print("object at location {}, {}, {}, {}".format( \
        object.rectangle.x, object.rectangle.x + object.rectangle.w, \
        object.rectangle.y, object.rectangle.y + object.rectangle.h))

===== Detect Objects - local =====
Detecting objects in local image:
object at location 1022, 2661, 744, 1948


## FUNCTION

In [72]:
def get_tags(filepath):
    local_image = open(filepath, "rb")

    tags_result = computervision_client.tag_image_in_stream(local_image)
    tags = tags_result.tags
    tags_name = []
    tags_confidence = []

    for tag in tags:
        tags_name.append(tag.name)
        tags_confidence.append(tag.confidence)
    
    return tags_name
    

In [73]:
local_image_path = 'sample01.jpg'

get_tags(local_image_path)

['tableware',
 'food',
 'baked goods',
 'plate',
 'drink',
 'coffee cup',
 'dishware',
 'saucer',
 'snack',
 'serveware',
 'meal',
 'mug',
 'tea',
 'fast food',
 'breakfast',
 'fork',
 'kitchen utensil',
 'dish',
 'brunch',
 'platter',
 'dessert',
 'cup',
 'coffee',
 'indoor',
 'sitting',
 'table']

In [76]:
def detect_objects(filepath):
    local_image = open(filepath, 'rb')
    
    detect_objects_results = computervision_client.detect_objects_in_stream(local_image)
    objects = detect_objects_results.objects
    return objects

In [82]:
objects = detect_objects(local_image_path)
objects[0].rectangle.x


879